In [1]:
# !python ./chainlink_stuffs/ingest.py 

In [1]:
import pickle
from IPython.display import Markdown
from langchain.docstore.document import Document

In [2]:
with open('../algovate/data/documents.pkl', 'rb') as f:
    documents = pickle.load(f)

### Retrieving from stackoverflow

step1 : https://stackoverflow.com/oauth?client_id=26442&scope=read_inbox&redirect_uri=https://stackapps.com

step2: WzYapLyZRU2(L5smk9jrzQ))

Step3 : !curl -d "client_id=26442&client_secret=1*L23oud4Rorl*sgpP86uQ((&code=WzYapLyZRU2(L5smk9jrzQ))&redirect_uri=https://stackapps.com" -X POST https://stackoverflow.com/oauth/access_token

step4: access_token=StCiXPrq6YvuE0AJiUM5TA))

In [50]:
import re
import time
import json
import requests
import html2text
from datetime import datetime

In [66]:
#https://api.stackexchange.com/docs/questions
def get_questions(tag, page, pagesize, from_date, to_date, key, access_token):
    # url = f'https://api.stackexchange.com/2.3/questions?order=desc&sort=creation&tagged={tag}&site={site}&pagesize={pagesize}&key={key}&access_token={access_token}&filter=withbody'
    url = f"https://api.stackexchange.com/2.3/questions?page={page}&pagesize={pagesize}&tagged={tag}&fromdate={from_date}&todate={to_date}&key={key}&access_token={access_token}&order=desc&sort=activity&site=stackoverflow&filter=withbody"
    response = requests.get(url)
    return response.json()['items']

def get_answers(question_id, site, key, access_token):
    url = f'https://api.stackexchange.com/2.3/questions/{question_id}/answers?order=desc&sort=activity&site=stackoverflow&key={key}&access_token={access_token}&filter=withbody'
    response = requests.get(url)
    return response.json()['items']


In [67]:
key = 'xbyZm8lZ94ZKKl0aQ5iYhA(('
access_token = 'StCiXPrq6YvuE0AJiUM5TA))'
tag = 'chainlink'
pagesize = 99
from_date = "1609459200"
to_date = "1685577600"

all_questions = None
for i in range(1, 5):
    page = i
    if all_questions is None:
        all_questions = get_questions(
            tag=tag, pagesize=pagesize, key=key, 
            access_token=access_token, from_date=from_date, 
            to_date=to_date, page=page
        )
    else:
        all_questions.extend(get_questions(
            tag=tag, pagesize=pagesize, key=key, 
            access_token=access_token, from_date=from_date, 
            to_date=to_date, page=page
            )
        )

    time.sleep(5)

In [68]:
all_questions = sorted(all_questions, key=lambda x: x['creation_date'], reverse=True)

In [69]:
h = html2text.HTML2Text()
h.ignore_links = False

questions_and_answers = {}

for question in all_questions:
    answers = get_answers(question['question_id'], site, key, access_token)
    if not answers:  # Skip questions with no answers
        continue
    question_body = re.sub(r'!\[.*?\]\(.*?\)', '', h.handle(question['body']))
    question_str = "Question: (Asked on: " + datetime.utcfromtimestamp(question['creation_date']).strftime('%Y-%m-%d %H:%M:%S') + ")\n" + question['title'] + "\n"
    question_url = "URL: " + question['link'] + "\n\n"
    question_body_str = "Question Body:\n" + question_body + "\n\n"
    answers_str = "Answers:\n" + "\n---\n".join(["(Answered on: " + datetime.utcfromtimestamp(answer['creation_date']).strftime('%Y-%m-%d %H:%M:%S') + ")\n" + re.sub(r'!\[.*?\]\(.*?\)', '', h.handle(answer['body'])) for answer in answers]) + "\n\n"
    text = question_str + question_url + question_body_str + answers_str

    questions_and_answers[question['question_id']] = Document(page_content=text, metadata={"source":question['link']})

In [72]:
with open("stackoverflow.pkl", "wb") as f:
    pickle.dump(questions_and_answers, f)

In [73]:
stack_overflow_documents = []
for k,v in questions_and_answers.items():
    stack_overflow_documents.append(v)

### Chainlink Education

In [74]:
import fnmatch
import tempfile
import subprocess
from pathlib import Path
from typing import List, Tuple, Optional
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

In [75]:
def load_gh_data( 
    gh_tuple:tuple, 
    chunk: bool = True, 
    chunk_size: int = 1200
):
    repo_url = gh_tuple[0]
    wildcards = gh_tuple[1]
    documents = get_github_docs(repo_url, wildcards) 
    
    # splitter = RecursiveCharacterTextSplitter(
    #     chunk_size = 1200,
    #     chunk_overlap  = 20,
    # )
    # documents = splitter.split_documents(documents)

    return documents

def get_github_docs(
    repo_url: str,
    wildcards: Optional[List[str]] = None
):
    repo_url = repo_url.replace('.git', '')
    url_parts = repo_url.split('/')
    if len(url_parts) < 5 or not url_parts[2].endswith("github.com"):
        raise ValueError("Invalid GitHub URL format")

    repo_owner = url_parts[3]
    repo_name = url_parts[4]

    if len(url_parts) > 6 and url_parts[5] == 'tree':
        branch = '/'.join(url_parts[6:])
    else:
        branch = None

    repo_url = f"https://github.com/{repo_owner}/{repo_name}"
    if not repo_url.endswith(".git"):
        repo_url += ".git"

    with tempfile.TemporaryDirectory() as d:
        if branch is not None:
            git_command = f"git clone --depth 1 -b {branch} {repo_url} ."
        else:
            git_command = f"git clone --depth 1 {repo_url} ."

        subprocess.check_call(
            git_command,
            cwd=d,
            shell=True,
        )
        git_sha = (
            subprocess.check_output("git rev-parse HEAD", shell=True, cwd=d)
            .decode("utf-8")
            .strip()
        )

        repo_path = Path(d)
        markdown_files = list(repo_path.glob("**/*.md")) + list(
            repo_path.glob("**/*.mdx")
        )
        if wildcards is not None:
            wildcards = [f"*{pattern}/*" if  not pattern.startswith('*') and not pattern.endswith('/*') else pattern for pattern in wildcards]
            filtered_files = []
            for wildcard in wildcards:
                filtered_files.extend(
                    file for file in markdown_files if fnmatch.fnmatch(str(file), wildcard)
                )
            markdown_files = list(set(filtered_files))  # Remove duplicates

        documents = []
        for markdown_file in markdown_files:
            with open(markdown_file, "r") as f:
                relative_path = markdown_file.relative_to(repo_path)
                github_url = f"https://github.com/{repo_owner}/{repo_name}/blob/{git_sha}/{relative_path}"
                read = f.read()
                documents.append(Document(page_content=read, metadata={"source": github_url}))

    return documents

In [76]:
gh_tuple = ("https://github.com/oceanByte/chainlink-education.git", "src/api/src/shared/course")

chainlink_education_documents = load_gh_data(gh_tuple)

Cloning into '.'...


### Merge all three into one document

In [77]:
len(documents), len(stack_overflow_documents), len(chainlink_education_documents)

(151, 309, 306)

In [78]:
overall_documents = documents + stack_overflow_documents + chainlink_education_documents
len(overall_documents)

766

In [43]:
with open("../algovate/data/combined_documents.pkl", "wb") as f:
    pickle.dump(overall_documents, f)